In [0]:
from pyspark.sql import functions as F

catalog       = "smartdavid_catalog"
schema_silver = "silver"
schema_golden = "golden"

In [0]:
df_pagos_gestion = spark.table(f"{catalog}.{schema_silver}.pagos_con_gestion")
df_united_clean  = spark.table(f"{catalog}.{schema_silver}.united_data_clean")
df_pagos_all     = spark.table(f"{catalog}.{schema_silver}.pagos_consolidado")

print(f"pagos_con_gestion : {df_pagos_gestion.count()} registros")
print(f"united_data_clean : {df_united_clean.count()} registros")
print(f"pagos_consolidado : {df_pagos_all.count()} registros")

In [0]:
df_rendimiento = (
    df_pagos_gestion
    .filter(F.col("tiene_gestion") == True)
    .groupBy("colaborador", "fuente_cartera")
    .agg(
        F.count("*").alias("total_pagos_gestionados"),
        F.round(F.sum("pago_total"), 2).alias("monto_total_recaudado"),
        F.round(F.avg("pago_total"), 2).alias("promedio_pago"),
        F.countDistinct("pag_numruc").alias("contribuyentes_unicos")
    )
    .orderBy(F.col("monto_total_recaudado").desc())
)

print(f"Rendimiento por colaborador: {df_rendimiento.count()} filas")
df_rendimiento.display()

In [0]:
df_pagos_mensual = (
    df_pagos_gestion
    .filter(F.col("crt_fecpag").isNotNull())
    .withColumn("anio", F.year("crt_fecpag"))
    .withColumn("mes",  F.month("crt_fecpag"))
    .withColumn("mes_nombre", F.date_format("crt_fecpag", "MMMM"))
    .groupBy("anio", "mes", "mes_nombre", "fuente_cartera")
    .agg(
        F.count("*").alias("cantidad_pagos"),
        F.round(F.sum("pago_total"), 2).alias("monto_total"),
        F.round(F.avg("pago_total"), 2).alias("promedio_pago"),
        F.sum(F.when(F.col("tiene_gestion") == True, 1).otherwise(0)).alias("pagos_gestionados")
    )
    .orderBy("anio", "mes")
)

print(f"Pagos por mes: {df_pagos_mensual.count()} filas")
df_pagos_mensual.display()

In [0]:
# Contribuyentes que NO han pagado ninguna cuota
df_pendientes = (
    df_united_clean
    .join(
        df_pagos_all.select("valor").distinct(),
        df_united_clean["num_resol"] == df_pagos_all["valor"],
        how="left_anti"
    )
    .select(
        "ruc", "nombre_contribuyente", "colaborador",
        "departamento", "estado", "saldo",
        "fecha_vencimiento", "tipo_fraccionamiento"
    )
    .withColumn("saldo", F.round(F.col("saldo"), 2))
    .orderBy(F.col("saldo").desc())
)

total_pendiente = df_pendientes.count()
saldo_total     = df_pendientes.agg(F.sum("saldo")).collect()[0][0]

print(f"Contribuyentes sin pago  : {total_pendiente}")
print(f"Saldo total por recuperar: S/ {saldo_total:,.2f}")
df_pendientes.limit(5).display()

In [0]:
# Cell 6 — Resumen por Departamento
df_resumen_depto = (
    df_pagos_gestion
    .filter(F.col("tiene_gestion") == True)
    .join(
        df_united_clean.select(
            F.col("num_resol"),
            F.col("departamento").alias("depto_united")
        ),
        df_pagos_gestion["valor"] == df_united_clean["num_resol"],
        how="left"
    )
    .groupBy("depto_united", "fuente_cartera")
    .agg(
        F.count("*").alias("total_pagos"),
        F.round(F.sum("pago_total"), 2).alias("monto_recaudado"),
        F.countDistinct("pag_numruc").alias("contribuyentes")
    )
    .withColumnRenamed("depto_united", "departamento")
    .orderBy(F.col("monto_recaudado").desc())
)

print(f"Resumen por departamento: {df_resumen_depto.count()} filas")
df_resumen_depto.display()

In [0]:
# Tabla 1: Rendimiento colaborador
df_rendimiento.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog}.{schema_golden}.rendimiento_colaborador")

# Tabla 2: Pagos por mes
df_pagos_mensual.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog}.{schema_golden}.pagos_mensual")

# Tabla 3: Cartera pendiente
df_pendientes.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog}.{schema_golden}.cartera_pendiente")

# Tabla 4: Resumen por departamento
df_resumen_depto.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog}.{schema_golden}.resumen_departamento")

print("Las 4 tablas Golden guardadas exitosamente")

In [0]:
%sql
SELECT 'rendimiento_colaborador' AS tabla, COUNT(*) AS registros 
FROM smartdavid_catalog.golden.rendimiento_colaborador
UNION ALL
SELECT 'pagos_mensual'           AS tabla, COUNT(*) AS registros 
FROM smartdavid_catalog.golden.pagos_mensual
UNION ALL
SELECT 'cartera_pendiente'       AS tabla, COUNT(*) AS registros 
FROM smartdavid_catalog.golden.cartera_pendiente
UNION ALL
SELECT 'resumen_departamento'    AS tabla, COUNT(*) AS registros 
FROM smartdavid_catalog.golden.resumen_departamento;

In [0]:
%sql
-- Top colaboradores por recaudación
SELECT 
    colaborador,
    fuente_cartera,
    total_pagos_gestionados,
    contribuyentes_unicos,
    monto_total_recaudado
FROM smartdavid_catalog.golden.rendimiento_colaborador
ORDER BY monto_total_recaudado DESC;